In [35]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import datetime
import csv
from scipy import signal

In [36]:
get_datetime = lambda s: datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")
def string_to_dt(time):
    '''
    This converts an input array of strings to a list of .datetimes
    '''
    dt = []
    for index,item in enumerate(time):
        dt = np.append(get_datetime(item),dt)
    dt=dt[::-1]
    return dt

def nearestDate(dates,pivot,length):
    '''
    This function inputs two sets of datetime lists. 
    The PIVOT and LENGTH belong to the list that have the desired base and length.
    Returns a array of strings from DATES that are closest to PIVOT matching the length of PIVOT
    '''
    match = []
    for i in range(0,length):
        closest = min(dates, key=lambda x: abs(x - pivot[i]))
        match = np.append(str(closest),match)
    return match[::-1]

def get_indecies(old_time,new_time):
    '''
    Function compares adjusted time arrays and outputs the indecies that match.
    These indicies are used to match the other variables to the compared times.
    '''
    indemic = []
    for i in range(0,len(old_time)):
        pal = get_datetime(old_time[i])
        for j in range(0,len(new_time)):
            gal = get_datetime(new_time[j])
            if pal == gal:
                indemic = np.append(i,indemic)
            else:
                pass
    return indemic[::-1]

def relative_velocity(FS_WD,FS_WS,bearing,speed):
    '''
    Calculates relative velocities from arrays of wind direction, wind speed, 
    activity direction, and activity speed. Outputs relative velocity, 
    as well as u and v components of the corresponding wind and activity vectors.
    '''
    
    Wind_Dir = np.array(FS_WD) # degrees
    Wind_Spd = np.array(FS_WS)*(1609.34/3600) # conversion to m/s
    Act_Dir = np.array(bearing) # degrees
    Act_Spd = np.array(speed) #m/s
    
    rad_wind = (Wind_Dir-180)*(np.pi/180) # subtract 180 b/c reported direction is where wind is coming FROM
    rad_act = (Act_Dir)*(np.pi/180) # direction of activity
    u1 = Wind_Spd*np.sin(rad_wind)
    v1 = Wind_Spd*np.cos(rad_wind)
    u2 = Act_Spd*np.sin(rad_act)
    v2 = Act_Spd*np.cos(rad_act)
    vect1 = np.sqrt(u1**2+u2**2)
    vect2 = np.sqrt(v1**2+v2**2)
    alpha = []
    for i in range(0,len(Wind_Spd)):
        angle = np.arccos(((u1[i]*v1[i])+(u2[i]*v2[i])) / (vect1[i]*vect2[i]))
        alpha = np.append(angle,alpha)
    alpha = alpha[::-1]
    v_r = np.sqrt(Wind_Spd**2+Act_Spd**2 - (2*Wind_Spd*Act_Spd*np.cos(np.pi+alpha)))
    return v_r,u1,v1,u2,v2

In [37]:
def combine_all(gps_file,sck1_file,sck2_file,sck3_file,fs_file,output_file):
    '''
    This function inputs the data from the gps, the three scks, and the fs wx station
    and combines them into a new file with the times of the scks and fs nearest
    to each gps time. The outputed file will have all data nearest to each recored
    gps position so that it may be plotted using GIS. 
    '''
    # reading in files
    gps = pd.read_csv(gps_file)
    sck1 = pd.read_csv(sck1_file)
    sck2 = pd.read_csv(sck2_file)
    sck3 = pd.read_csv(sck3_file)
    fs = pd.read_csv(fs_file)
    print 'Files read'
    
    # gathering times from files
    gpstime = gps['GPStime']
    sck1time = sck1['SCK1time']
    sck2time = sck2['SCK2time']
    sck3time = sck3['SCK3time']
    fstime = fs['FStime']
    
    # converting strings to datetimes
    gps_dt = string_to_dt(gpstime)
    sck1_dt = string_to_dt(sck1time)
    sck2_dt = string_to_dt(sck2time)
    sck3_dt = string_to_dt(sck3time)
    fs_dt = string_to_dt(fstime)
    
    # finding the new times matching the gps
    sck1_new = nearestDate(sck1_dt,gps_dt,len(gpstime))
    sck2_new = nearestDate(sck2_dt,gps_dt,len(gpstime))
    sck3_new = nearestDate(sck3_dt,gps_dt,len(gpstime))
    fs_new = nearestDate(fs_dt,gps_dt,len(gpstime))
    print 'Times matched'
    
    # gathering the indecies used in nearestDate
    i_sck1 = get_indecies(sck1time,sck1_new)
    i_sck2 = get_indecies(sck2time,sck2_new)
    i_sck3 = get_indecies(sck3time,sck3_new)
    i_fs = get_indecies(fstime,fs_new)
    
    #calculating relative velocity
    v_r,uwind,vwind,uact,vact = relative_velocity(fs['FS_WD'][i_fs],fs['FS_WS'][i_fs],
                                                  gps['bearing(deg)'],gps['speed(m/s)'])
    print 'Calculated relative velocity'
    
    # detrending the NO2 data (if statements fill dummy dates with zeros)   
    if np.isnan(list(sck1['SCK1_NO2'][i_sck1])[0]) == False:
        sck1_no2_d = signal.detrend(sck1['SCK1_NO2'][i_sck1])+np.mean(sck1['SCK1_NO2'][i_sck1])
    else:
        sck1_no2_d = np.zeros_like(sck1['SCK1_NO2'][i_sck1])
    if np.isnan(list(sck2['SCK2_NO2'][i_sck2])[0]) == False:
        sck2_no2_d = signal.detrend(sck2['SCK2_NO2'][i_sck2])+np.mean(sck2['SCK2_NO2'][i_sck2])
    else:
        sck2_no2_d = np.zeros_like(sck2['SCK2_NO2'][i_sck2])
    if np.isnan(list(sck3['SCK3_NO2'][i_sck3])[0]) == False:
        sck3_no2_d = signal.detrend(sck3['SCK3_NO2'][i_sck3])+np.mean(sck3['SCK3_NO2'][i_sck3])
    else:
        sck3_no2_d = np.zeros_like(sck3['SCK3_NO2'][i_sck3])
    print 'Detrended NO2'
    
    # write to columns in .csv
    rows = zip(gpstime,sck1_new,sck2_new,sck3_new,fs_new, # time columns
               
               gps['lat(DD)'],gps['lon(DD)'],gps['bearing(deg)'],gps['speed(m/s)'],gps['distance(m)'], #gps columns
               
               sck1['SCK1_CO'][i_sck1],sck1['SCK1_NO2'][i_sck1],sck1_no2_d,sck1['SCK1_TC'][i_sck1], # sck1 columns
               sck1['SCK1_RH'][i_sck1],sck1['SCK1_LX'][i_sck1],sck1['SCK1_DB'][i_sck1],
               
               sck2['SCK2_CO'][i_sck2],sck2['SCK2_NO2'][i_sck2],sck2_no2_d,sck2['SCK2_TC'][i_sck2], # sck3 columns
               sck2['SCK2_RH'][i_sck2],sck2['SCK2_LX'][i_sck2],sck2['SCK2_DB'][i_sck2],

               sck3['SCK3_CO'][i_sck3],sck3['SCK3_NO2'][i_sck3],sck3_no2_d,sck3['SCK3_TC'][i_sck3], # sck3 columns
               sck3['SCK3_RH'][i_sck3],sck3['SCK3_LX'][i_sck3],sck3['SCK3_DB'][i_sck3],
               
               fs['FS_TF'][i_fs],fs['FS_RH'][i_fs],fs['FS_WS'][i_fs], # fs columns
               fs['FS_WD'][i_fs],fs['FS_UP'][i_fs],fs['FS_DN'][i_fs],
              
               v_r, uwind, vwind, uact, vact) # calculated columns

    with open(output_file, 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(['GPStime','SCK1time','SCK2time','SCK3time','FStime',
                         'lat(DD)','lon(DD)','bearing(deg)','speed(m/s)','distance(m)',
                         'SCK1_CO','SCK1_NO2','SCK1_NO2_det','SCK1_TC','SCK1_RH','SCK1_LX','SCK1_DB',
                         'SCK2_CO','SCK2_NO2','SCK2_NO2_det','SCK2_TC','SCK2_RH','SCK2_LX','SCK2_DB',
                         'SCK3_CO','SCK3_NO2','SCK3_NO2_det','SCK3_TC','SCK3_RH','SCK3_LX','SCK3_DB',
                         'FS_TF','FS_RH','FS_WS','FS_WD','FS_UP','FS_DN',
                         'V_r','uwind','vwind','uact','vact'])
        for i in rows:
            writer.writerow(i) 
    print 'Done writing file'

In [38]:
def combine_one(gps_file,sck1_file,fs_file,output_file):
    '''
    This function inputs the data from the gps, one sck, and the fs wx station
    and combines them into a new file with the times of the scks and fs nearest
    to each gps time. The outputed file will have all data nearest to each recored
    gps position so that it may be plotted using GIS. 
    '''
    # reading in files
    gps = pd.read_csv(gps_file)
    sck1 = pd.read_csv(sck1_file)
    fs = pd.read_csv(fs_file)
    print 'Files read'
    
    # gathering times from files
    gpstime = gps['GPStime']
    sck1time = sck1['SCK1time']
    fstime = fs['FStime']
    
    # converting strings to datetimes
    gps_dt = string_to_dt(gpstime)
    sck1_dt = string_to_dt(sck1time)
    fs_dt = string_to_dt(fstime)
    
    # finding the new times matching the gps
    sck1_new = nearestDate(sck1_dt,gps_dt,len(gpstime))
    fs_new = nearestDate(fs_dt,gps_dt,len(gpstime))
    print 'Times matched'
    
    # gathering the indecies used in nearestDate
    i_sck1 = get_indecies(sck1time,sck1_new)
    i_fs = get_indecies(fstime,fs_new)
    
    #calculating relative velocity
    v_r,uwind,vwind,uact,vact = relative_velocity(fs['FS_WD'][i_fs],fs['FS_WS'][i_fs],
                                                  gps['bearing(deg)'],gps['speed(m/s)'])
    print 'Calculated relative velocity'
    
    # detrending the NO2 data
    sck1_no2_d = signal.detrend(sck1['SCK1_NO2'][i_sck1])+np.mean(sck1['SCK1_NO2'][i_sck1])
    print 'Detrended NO2'
    
    # write to columns in .csv
    rows = zip(gpstime,sck1_new,fs_new, # time columns
               
               gps['lat(DD)'],gps['lon(DD)'],gps['bearing(deg)'], #gps columns
               gps['speed(m/s)'],gps['distance(m)'], 
               
               sck1['SCK1_CO'][i_sck1],sck1['SCK1_NO2'][i_sck1],sck1_no2_d,sck1['SCK1_TC'][i_sck1], # sck1 columns
               sck1['SCK1_RH'][i_sck1],sck1['SCK1_LX'][i_sck1],sck1['SCK1_DB'][i_sck1],
               
               fs['FS_TF'][i_fs],fs['FS_RH'][i_fs],fs['FS_WS'][i_fs], # fs columns
               fs['FS_WD'][i_fs],fs['FS_UP'][i_fs],fs['FS_DN'][i_fs],
    
               v_r, uwind, vwind, uact, vact) # calculated columns

    with open(output_file, 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(['GPStime','SCK1time','FStime',
                         'lat(DD)','lon(DD)','bearing(deg)','speed(m/s)','distance(m)',
                         'SCK1_CO','SCK1_NO2','SCK1_NO2_det','SCK1_TC','SCK1_RH','SCK1_LX','SCK1_DB',
                         'FS_TF','FS_RH','FS_WS','FS_WD','FS_UP','FS_DN',
                         'V_r','uwind','vwind','uact','vact'])
        for i in rows:
            writer.writerow(i)
    print 'Done writing file'

In [39]:
combine_one('GPS_rides/BEAR/Oct2.csv',
            'SCK_rides/RO/Oct2_SCK1_RO.csv',
            'FS_data/Resamples/Oct2.csv',
            'Combined/Oct2.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [40]:
combine_one('GPS_rides/BEAR/Oct14.csv',
            'SCK_rides/RO/Oct14_SCK1_RO.csv',
            'FS_data/Resamples/Oct14.csv',
            'Combined/Oct14.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [41]:
combine_one('GPS_rides/BEAR/Nov2.csv',
            'SCK_rides/RO/Nov2_SCK1_RO.csv',
            'FS_data/Resamples/Nov2.csv',
            'Combined/Nov2.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [42]:
combine_one('GPS_rides/BEAR/Nov3.csv',
            'SCK_rides/RO/Nov3_SCK1_RO.csv',
            'FS_data/Resamples/Nov3.csv',
            'Combined/Nov3.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [43]:
combine_one('GPS_rides/BEAR/Nov4.csv',
            'SCK_rides/RO/Nov4_SCK1_RO.csv',
            'FS_data/Resamples/Nov4.csv',
            'Combined/Nov4.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [44]:
combine_one('GPS_rides/BEAR/Nov10.csv',
            'SCK_rides/RO/Nov10_SCK1_RO.csv',
            'FS_data/Resamples/Nov10.csv',
            'Combined/Nov10.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [45]:
combine_one('GPS_rides/BEAR/Feb19.csv',
            'SCK_rides/RO/Feb19_SCK1_RO.csv',
            'FS_data/Resamples/Feb19.csv',
            'Combined/Feb19.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [46]:
combine_one('GPS_rides/BEAR/Feb20.csv',
            'SCK_rides/RO/Feb20_SCK1_RO.csv',
            'FS_data/Resamples/Feb20.csv',
            'Combined/Feb20.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [47]:
combine_all('GPS_rides/BEAR/Mar2.csv',
        'SCK_rides/RO/Mar2_SCK1_RO.csv',
        'SCK_rides/RO/Mar2_SCK2_RO.csv',
        'SCK_rides/RO/Mar2_SCK3_RO.csv',
        'FS_data/Resamples/Mar2.csv',
        'Combined/Mar2.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [48]:
combine_all('GPS_rides/BEAR/Mar4.csv',
        'SCK_rides/RO/Mar4_SCK1_RO.csv',
        'SCK_rides/RO/Mar4_SCK2_RO.csv',
        'SCK_rides/RO/Mar4_SCK3_RO.csv',
        'FS_data/Resamples/Mar4.csv',
        'Combined/Mar4.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [49]:
combine_all('GPS_rides/BEAR/Mar7.csv',
        'SCK_rides/RO/Mar7_SCK1_RO.csv',
        'SCK_rides/RO/Mar7_SCK2_RO.csv',
        'SCK_rides/RO/Mar7_SCK3_RO.csv',
        'FS_data/Resamples/Mar7.csv',
        'Combined/Mar7.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [50]:
combine_all('GPS_rides/BEAR/Mar14.csv',
        'SCK_rides/RO/Mar14_SCK1_RO.csv',
        'SCK_rides/RO/Mar14_SCK2_RO.csv',
        'SCK_rides/RO/Mar14_SCK3_RO.csv',
        'FS_data/Resamples/Mar14.csv',
        'Combined/Mar14.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [51]:
combine_all('GPS_rides/BEAR/Mar21.csv',
        'SCK_rides/RO/Mar21_SCK1_RO.csv',
        'SCK_rides/RO/Mar21_SCK2_RO.csv',
        'SCK_rides/RO/Mar21_SCK3_RO.csv',
        'FS_data/Resamples/Mar21.csv',
        'Combined/Mar21.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [52]:
combine_all('GPS_rides/BEAR/Mar22.csv',
        'SCK_rides/RO/Mar22_SCK1_RO.csv',
        'SCK_rides/RO/Mar22_SCK2_RO.csv',
        'SCK_rides/RO/Mar22_SCK3_RO.csv',
        'FS_data/Resamples/Mar22.csv',
        'Combined/Mar22.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [53]:
combine_all('GPS_rides/BEAR/Mar25.csv',
        'SCK_rides/RO/Mar25_SCK1_RO.csv',
        'SCK_rides/RO/Mar25_SCK2_RO.csv',
        'SCK_rides/RO/Mar25_SCK3_RO.csv',
        'FS_data/Resamples/Mar25.csv',
        'Combined/Mar25.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [54]:
combine_all('GPS_rides/BEAR/Mar28.csv',
        'SCK_rides/RO/Mar28_SCK1_RO.csv',
        'SCK_rides/RO/Mar28_SCK2_RO.csv',
        'SCK_rides/RO/Mar28_SCK3_RO.csv',
        'FS_data/Resamples/Mar28.csv',
        'Combined/Mar28.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [55]:
combine_all('GPS_rides/BEAR/Mar29.csv',
        'SCK_rides/RO/Mar29_SCK1_RO.csv',
        'SCK_rides/RO/Mar29_SCK2_RO.csv',
        'SCK_rides/RO/Mar29_SCK3_RO.csv',
        'FS_data/Resamples/Mar29.csv',
        'Combined/Mar29.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [56]:
combine_all('GPS_rides/BEAR/Mar30.csv',
        'SCK_rides/RO/Mar30_SCK1_RO.csv',
        'SCK_rides/RO/Mar30_SCK2_RO.csv',
        'SCK_rides/RO/Mar30_SCK3_RO.csv',
        'FS_data/Resamples/Mar30.csv',
        'Combined/Mar30.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [57]:
combine_all('GPS_rides/BEAR/Apr4.csv',
        'SCK_rides/RO/Apr4_SCK1_RO.csv',
        'SCK_rides/RO/Apr4_SCK2_RO.csv',
        'SCK_rides/RO/Apr4_SCK3_RO.csv',
        'FS_data/Resamples/Apr4.csv',
        'Combined/Apr4.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [58]:
combine_all('GPS_rides/BEAR/Apr5.csv',
        'SCK_rides/RO/Apr5_SCK1_RO.csv',
        'SCK_rides/RO/Apr5_SCK2_RO.csv',
        'SCK_rides/RO/Apr5_SCK3_RO.csv',
        'FS_data/Resamples/Apr5.csv',
        'Combined/Apr5.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [59]:
combine_all('GPS_rides/BEAR/Apr6.csv',
        'SCK_rides/RO/Apr6_SCK1_RO.csv',
        'SCK_rides/RO/Apr6_SCK2_RO.csv',
        'SCK_rides/RO/Apr6_SCK3_dummy.csv',
        'FS_data/Resamples/Apr6.csv',
        'Combined/Apr6.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [60]:
combine_all('GPS_rides/BEAR/Apr7.csv',
        'SCK_rides/RO/Apr7_SCK1_RO.csv',
        'SCK_rides/RO/Apr7_SCK2_RO.csv',
        'SCK_rides/RO/Apr7_SCK3_RO.csv',
        'FS_data/Resamples/Apr7.csv',
        'Combined/Apr7.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [61]:
combine_all('GPS_rides/BEAR/Apr13.csv',
        'SCK_rides/RO/Apr13_SCK1_RO.csv',
        'SCK_rides/RO/Apr13_SCK2_RO.csv',
        'SCK_rides/RO/Apr13_SCK3_RO.csv',
        'FS_data/Resamples/Apr13.csv',
        'Combined/Apr13.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [62]:
combine_all('GPS_rides/BEAR/Apr14.csv',
        'SCK_rides/RO/Apr14_SCK1_RO.csv',
        'SCK_rides/RO/Apr14_SCK2_RO.csv',
        'SCK_rides/RO/Apr14_SCK3_RO.csv',
        'FS_data/Resamples/Apr14.csv',
        'Combined/Apr14.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [63]:
combine_all('GPS_rides/BEAR/Apr15.csv',
        'SCK_rides/RO/Apr15_SCK1_RO.csv',
        'SCK_rides/RO/Apr15_SCK2_RO.csv',
        'SCK_rides/RO/Apr15_SCK3_RO.csv',
        'FS_data/Resamples/Apr15.csv',
        'Combined/Apr15.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [64]:
combine_all('GPS_rides/BEAR/Apr27.csv',
        'SCK_rides/RO/Apr27_SCK1_dummy.csv',
        'SCK_rides/RO/Apr27_SCK2_RO.csv',
        'SCK_rides/RO/Apr27_SCK3_RO.csv',
        'FS_data/Resamples/Apr27.csv',
        'Combined/Apr27.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [65]:
combine_all('GPS_rides/BEAR/Apr28.csv',
        'SCK_rides/RO/Apr28_SCK1_RO.csv',
        'SCK_rides/RO/Apr28_SCK2_RO.csv',
        'SCK_rides/RO/Apr28_SCK3_RO.csv',
        'FS_data/Resamples/Apr28.csv',
        'Combined/Apr28.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [66]:
combine_all('GPS_rides/BEAR/Apr29.csv',
        'SCK_rides/RO/Apr29_SCK1_RO.csv',
        'SCK_rides/RO/Apr29_SCK2_RO.csv',
        'SCK_rides/RO/Apr29_SCK3_RO.csv',
        'FS_data/Resamples/Apr29.csv',
        'Combined/Apr29.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [67]:
combine_all('GPS_rides/BEAR/May5.csv',
        'SCK_rides/RO/May5_SCK1_dummy.csv',
        'SCK_rides/RO/May5_SCK2_RO.csv',
        'SCK_rides/RO/May5_SCK3_RO.csv',
        'FS_data/Resamples/May5.csv',
        'Combined/May5.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file


In [68]:
combine_all('GPS_rides/BEAR/May6.csv',
        'SCK_rides/RO/May6_SCK1_dummy.csv',
        'SCK_rides/RO/May6_SCK2_RO.csv',
        'SCK_rides/RO/May6_SCK3_RO.csv',
        'FS_data/Resamples/May6.csv',
        'Combined/May6.csv')

Files read
Times matched
Calculated relative velocity
Detrended NO2
Done writing file
